# Datan pika-analyysi Pythonilla

In [1]:
import pandas as pd

#Avaa data
df=pd.read_excel('http://www.elisanet.fi/taanila/data2.xlsx',sheet_name='Data')
df.head()

,nro,sukupuoli,ikä,perhe,koulutus,palvelusvuodet,palkka,tyytyväisyys johtoon,tyytyväisyys työtovereihin,tyytyväisyys työympäristöön,tyytyväisyys palkkaan,tyytyväisyys työtehtäviin,työterveyshuolto,lomaosake,kuntosali,hieroja
0,1,mies,38,perheetön,1 peruskoulu,22.0,3587,3,3.0,3,3,3,NaN,NaN,NaN,NaN
1,2,mies,29,perheellinen,2 2. aste,10.0,2963,1,5.0,2,1,3,NaN,NaN,NaN,NaN
2,3,mies,30,perheetön,1 peruskoulu,7.0,1989,3,4.0,1,1,3,1.0,NaN,NaN,NaN
3,4,mies,36,perheellinen,1 peruskoulu,14.0,2144,3,3.0,3,3,3,1.0,NaN,NaN,NaN
4,5,mies,24,perheetön,2 2. aste,4.0,2183,2,3.0,2,1,2,1.0,NaN,NaN,NaN


In [2]:
#Määrittele mitkä datan muuttujista ovat kategorisia, mielipideasteikollisia tai määrällisiä
#Jos tietyn tyyppisiä muuttujia ei ole, niin käytä tyhjää listaa, esimerkiksi kvantit=[]
kategoriset=['sukupuoli','perhe','koulutus']
mielipiteet=['tyytyväisyys johtoon', 'tyytyväisyys työtovereihin',
        'tyytyväisyys työympäristöön','tyytyväisyys palkkaan','tyytyväisyys työtehtäviin']
kvantit=['ikä','palvelusvuodet','palkka']

In [4]:
#Avaa/luo tiedosto, johon tulokset kirjoitetaan
#Tiedoston voit avata myöhemmin esimerkiksi Exceliin
f=open('example1.csv', 'a')

#Kategoristen muuttujien frekvenssitaulukot
if kategoriset:
    for var in kategoriset:
        df1=pd.crosstab(df[var],'lkm')
        df1['prosenttia'] = df1/df1.sum()
        df1.loc['Yhteensä'] = df1.sum()
        df1.to_csv(f, sep=';', decimal=',')
        f.write('\n')

#Mielipideasteikollisten muuttujien frekvenssitaulukot
if mielipiteet:
    for var in mielipiteet:
        df1=pd.crosstab(df[var],'lkm')    
        df1['prosenttia'] = df1/df1.sum()
        df1.loc['Yhteensä'] = df1.sum()
        df1.to_csv(f, sep=';', decimal=',')
        f.write('\n')

#Ristiintaulukoinnit kategoristen muuttujien välillä
if kategoriset:
    for var1 in kategoriset:
        for var2 in kategoriset:
            if var1!=var2:
                df1=pd.crosstab(df[var2],df[var1])
                df1.loc['Yhteensä']=df1.sum()
                df2=pd.crosstab(df[var2],df[var1],normalize='columns')
                df2.index.name=var2+'/'+var1
                df2.loc['Yhteensä']=df2.sum()
                df2.loc['n']=df1.loc['Yhteensä']
                df2.to_csv(f, sep=';', decimal=',')
                f.write('\n')

#Kategoristen ja mielipideasteikollisten väliset ristiintaulukoinnit
if kategoriset and mielipiteet:
    for var1 in kategoriset:
        for var2 in mielipiteet:
            df1=pd.crosstab(df[var2],df[var1])
            df1.loc['Yhteensä']=df1.sum()
            df2=pd.crosstab(df[var2],df[var1],normalize='columns')
            df2.index.name=var2+'/'+var1
            df2.loc['Yhteensä']=df2.sum()
            df2.loc['n']=df1.loc['Yhteensä']
            df2.to_csv(f, sep=';', decimal=',')
            f.write('\n')

#Mielipiteiden tunnusluvut
if mielipiteet:
    df1=df[mielipiteet].describe()
    df1.transpose().to_csv(f, sep=';', decimal=',')
    f.write('\n')

#Mielipiteiden tunnusluvut kategoristen määräämissä ryhmissä
if kategoriset and mielipiteet:
    for var1 in kategoriset:
        for var2 in mielipiteet:
            df1=df.groupby(var1)[var2].describe()
            df1.index.name=var2+'/'+var1
            df1.to_csv(f, sep=';', decimal=',')
            f.write('\n')

#Määrällisten muuttujien tunnusluvut
if kvantit:
    df1=df[kvantit].describe()
    df1.to_csv(f, sep=';', decimal=',')
    f.write('\n')

#Määrällisten tunnusluvut kategoristen määräämissä ryhmissä
if kategoriset and kvantit:
    for var1 in kategoriset:
        for var2 in kvantit:
            df1=df.groupby(var1)[var2].describe()
            df1.index.name=var2+'/'+var1
            df1.to_csv(f, sep=';', decimal=',')
            f.write('\n')

#Määrällisten völiset korrelaatiot
if kvantit:
    df1=df[kvantit].corr()
    df1.index.name='Korrelaatiot'
    df1.to_csv(f, sep=';', decimal=',')
    f.write('\n')

#Sulje tiedosto
f.close()

In [5]:
#Määrittele samaan monivalintakysymykseen liittyvät vaihtoehdot
#Jos sinulla on useita monivalintakysymyksiä, niin tämä lohko pitää toistaa jokaiselle monivalintakysymykselle
monivalinnat=['työterveyshuolto','lomaosake','kuntosali','hieroja']

#Avaa tiedosto, johon tulokset kirjoitetaan
f=open('example2.csv', 'a')

#Valinnat lukumäärinä ja prosentteina kyselyn vastaajista
df1 = df[monivalinnat].sum().sort_values(ascending = False).to_frame()
df1 = df1.rename(columns = {0: 'lkm'})
df1['% vastaajista'] = df1['lkm'] / df.shape[0]
df1.loc['n','% vastaajista']=df.shape[0]
df1.to_csv(f, sep=';', decimal=',')
f.write('\n')

#Valinnat kategoristen määräämissä ryhmissä lukumäärinä ja prosentteina ryhmästä
for var1 in kategoriset:
    df1 = df.groupby(var1)[monivalinnat].sum()
    df1.to_csv(f, sep=';', decimal=',')
    f.write('\n')
    for value in df[var1].dropna().unique():
        value_total=df[var1].value_counts()[value]
        df1.loc[value]=df1.loc[value]/value_total
        df1.loc[value, 'n']=value_total
    df1.to_csv(f, sep=';', decimal=',')
    f.write('\n')
    
#Sulje tiedosto
f.close()